# Mapping Geotagged Images  v1.0

This script requires JPG images with geotagged information in the header, stored in a Dropbox folder that is publicly shared and an ArcGIS Online account capable of publishing services and maps.

To use:
1. Code block 1, line 20: Point to a local Dropbox folder with photos.  Photos can be in subdirectories.  Non-JPG files are ignored. A CSV file is gernated int he folder and used to create the ArcGIS Online layer. 
3. From the Dropbox.com website, share your folder and copy the shared folder URL. 
4. Code block 1, line 6: Add "&preview=" to the end of the URL and insert into script below (baseDropBoxSharefolder=).  It will look like: https://www.dropbox.com/sh/6icw00y0uuuiszc/AADIl61HVsZuzc5i3MuPZfv1a?dl=0&preview=
5. Code block 2, line 5: Add your ArcGIS Online username, password and subdomain.
6. A sample map output: https://edteam.maps.arcgis.com/home/webmap/viewer.html?webmap=561f05703fc8456d8ceb9240eeaf7e62
7. Access your AGO account after running this script to find your new layer.  Createa new map and add the layer to it. Set sharing as needed.

-Tom, October 2018
        

In [ ]:
# IMPORTS AND DEFINITIONS  ------------------------------------------------------
import pathlib
from PIL.ExifTags import TAGS
from PIL import Image
temp = "filename, lat, long, URL \r\n"
baseDropBoxSharefolder=""

# FUNCTIONS ---------------------------------------------------------------------
def get_exif(fn):
    ret = {}
    output=""
    i = Image.open(fn)
    info = i._getexif()
    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        ret[decoded] = value
    return ret

# PROCEEDURE  ---------------------------------------------------------------------
currentDirectory = pathlib.Path('')
currentPattern = "**/*.*"   
    #double ## (above) enables path to recurively search subdirs

# read the directory, list matching files, and call EXIF function
for currentFile in currentDirectory.glob(currentPattern):  
        # .name returns only filename, else full path returned
    if ((currentFile.suffix == '.jpg') or (currentFile.suffix == '.JPG')) :
        ret = get_exif(currentFile)
        try:
            print(currentFile.name)
            GPSdict = ret['GPSInfo']
            dlat=GPSdict[2][0][0]/GPSdict[2][0][1]
            mlat=(GPSdict[2][1][0]/GPSdict[2][1][1])/60
            slat=(GPSdict[2][2][0]/GPSdict[2][2][1])/3600
            dlon=GPSdict[4][0][0]/GPSdict[4][0][1]
            mlon=(GPSdict[4][1][0]/GPSdict[4][1][1])/60
            slon=(GPSdict[4][2][0]/GPSdict[4][2][1])/3600            
            if GPSdict[1] == 'N':
                lat=dlat+mlat+slat
            else:
                lat=(dlat+mlat+slat)*-1
            if GPSdict[3] == 'E':
                lon=dlon+mlon+slon
            else:  
                lon=(dlon+mlon+slon)*-1
            print(lat)
            print(lon)
            URL = baseDropBoxSharefolder + currentFile.name
            temp = temp + currentFile.name + "," + str(lat) + "," + str(lon)+ "," + URL + "\r\n"
        except:
           print('JPG file but no GPS') 
        print('--------------- ')
        print('  ')

#WRITE OUT CSV FILE ---------------------------------------------------------------------
w = open("output/dataout.txt","w+")
w.write(temp)
w.close()
print("Complete.")

In [ ]:
print("This may take a few minutes. Keep window open until you see the completion message (or an error).")

# CREATE AGO SERVICE ---------------------------------------------------------------------
from arcgis.gis import GIS
gis = GIS("https://<subdomain>.maps.arcgis.com", "<user>", "<password>")
csv_file = 'output/dataout.txt'
csv_properties={'title':'My Geotagged Images',
                'description':'Photographs that I have taken.',
                'tags':'photography, images, pictures', 'type':'CSV'}
thumbnail_path = ""
csv_item = gis.content.add(item_properties=csv_properties, data=csv_file,
                                     thumbnail = thumbnail_path)

csv_lyr = csv_item.publish(None)
csv_item.share(everyone=True, org=True)
csv_lyr.share(everyone=True, org=True)
print('*****  *****  *****  *****  *****')
print("AGO layer build complete.  Go look in your AGO root directory for your new layer. It should already be publicly shared.")
